In [33]:
import pandas as pd
import numpy as np
#from sklearn import train_test_split
import sklearn.metrics
import dataclasses
import lightgbm as lgb
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

In [22]:
train_processed_df = pd.read_pickle("inputs/train_processed.pkl")

In [ ]:
# search_space = {

# }

# pbt_scheduler = PopulationBasedTraining(
#         time_attr='time_total_s',
#         metric='mean_accuracy',
#         mode='max',
#         perturbation_interval=600.0,
#         hyperparam_mutations={
#             "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
#             "alpha": lambda: random.uniform(0.0, 1.0),
#         ...
#         })
# tune.run( train_processed_df, scheduler=pbt_scheduler)

In [12]:
from sklearn.model_selection import train_test_split
X=train_processed_df.drop('diabetes_mellitus', axis=1)
y=train_processed_df.diabetes_mellitus
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=314, stratify=y)
train_set = lgb.Dataset(X_train, label=y_train)
test_set = lgb.Dataset(X_test, label=y_test)

In [17]:
def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})

In [35]:
def train_diabetes(config):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    #train_processed_df = pd.read_pickle("inputs/train_processed.pkl")
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.25)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        mean_accuracy=sklearn.metrics.roc_auc_score(test_y, pred_labels),
        done=True)


In [36]:
config = {
        "objective": "binary",
        #"metric": "binary_error",
        "metric": "mean_accuracy",
        "verbose": -1,
        "boosting_type": tune.grid_search(["gbdt", "dart"]),
        "num_leaves": tune.randint(10, 1000),
        "learning_rate": tune.loguniform(1e-2, 1e-1)
    }
analysis = tune.run(
        train_diabetes,
        metric="auc",
        mode="min",
        config=config,
        num_samples=2, #Number of times to sample from the hyperparameter space
        #scheduler=ASHAScheduler()
        local_dir="/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/logging",
        scheduler=PopulationBasedTraining()
        )

print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 15:03:27,518	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149703 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 15:03:27,646	WARNING util.py:152 -- The `start_trial` operation took 0.858 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,learning_rate,num_leaves
train_diabetes_17355_00000,RUNNING,,gbdt,0.0200876,439


2021-03-18 15:03:28,892	WARNING util.py:152 -- The `start_trial` operation took 0.679 s, which may be a performance bottleneck.
2021-03-18 15:03:30,083	WARNING util.py:152 -- The `start_trial` operation took 0.733 s, which may be a performance bottleneck.
2021-03-18 15:03:31,472	WARNING util.py:152 -- The `start_trial` operation took 0.765 s, which may be a performance bottleneck.
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=35848)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
(pid=35848)   warnings.warn('{} in param dict is overridden.'.format(cat_alias))
(pid=35863) /Users/anitaclement/

nt/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=35863)     output = fn()
(pid=35863)   File "<ipython-input-35-e67547cfe31f>", line 15, in train_diabetes
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/engine.py", line 267, in train
(pid=35863)     evaluation_result_list=evaluation_result_list))
(pid=35863)   File "<ipython-input-17-c4e673d45b99>", line 3, in LightGBMCallback
(pid=35863) IndexError: list index out of range
(pid=35863) Exception in thread Thread-2:
(pid=35863) Traceback (most recent call last):
(pid=35863)   File "/Users/anitaclement/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=35863)     self.run()
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/t

TuneError: ('Trials did not complete', [train_diabetes_17355_00000, train_diabetes_17355_00001, train_diabetes_17355_00002, train_diabetes_17355_00003])

In [42]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from ray import tune
from ray.tune.schedulers import ASHAScheduler


def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})


def train_breast_cancer(config):
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.25)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        #mean_accuracy=sklearn.metrics.accuracy_score(test_y, pred_labels),
        #binary_error=sklearn.metrics.accuracy_score(test_y, pred_labels),
        binary_error=sklearn.metrics.auc_score(test_y, pred_labels),
        done=True)


if __name__ == "__main__":
    config = {
        "objective": "binary",
        "metric": "binary_error",
        "verbose": -1,
        "boosting_type": tune.grid_search(["gbdt", "dart"]),
        "num_leaves": tune.randint(10, 1000),
        "learning_rate": tune.loguniform(1e-2, 1e-1)
    }

    analysis = tune.run(
        train_breast_cancer,
        metric="binary_error",
        mode="min",
        config=config,
        num_samples=2,
        #scheduler=ASHAScheduler())
        scheduler=PopulationBasedTraining())

    print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 15:31:32,718	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149673 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 15:31:32,857	WARNING util.py:152 -- The `start_trial` operation took 0.965 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,learning_rate,num_leaves
train_breast_cancer_039ee_00000,RUNNING,,gbdt,0.00018766,484


2021-03-18 15:31:34,581	WARNING util.py:152 -- The `start_trial` operation took 0.969 s, which may be a performance bottleneck.
2021-03-18 15:31:36,105	WARNING util.py:152 -- The `start_trial` operation took 0.838 s, which may be a performance bottleneck.
2021-03-18 15:31:38,651	WARNING util.py:152 -- The `start_trial` operation took 1.494 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,learning_rate,num_leaves
train_breast_cancer_039ee_00000,RUNNING,,gbdt,0.00018766,484
train_breast_cancer_039ee_00001,RUNNING,,dart,0.000536684,395
train_breast_cancer_039ee_00002,RUNNING,,gbdt,4.10828e-07,934
train_breast_cancer_039ee_00003,RUNNING,,dart,0.0449364,558


(pid=36304) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=36304)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=36305) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=36305)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=36304) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
(pid=36304)   warnings.warn('{} in param dict is overridden.'.format(cat_alias))
(pid=36305) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-package

...

Trial name,status,loc,boosting_type,learning_rate,num_leaves,iter,total time (s),binary_error
train_breast_cancer_039ee_00002,RUNNING,192.168.0.4:36342,gbdt,4.10828e-07,934,96,115.694,0.216779
train_breast_cancer_039ee_00000,TERMINATED,,gbdt,0.00018766,484,101,114.072,0.781592
train_breast_cancer_039ee_00001,TERMINATED,,dart,0.000536684,395,101,96.5724,0.785771
train_breast_cancer_039ee_00003,TERMINATED,,dart,0.0449364,558,101,113.198,0.831254


Result for train_breast_cancer_039ee_00002:
  binary_error: 0.21677934849416103
  date: 2021-03-18_15-33-37
  done: false
  experiment_id: bbd963254b9740aaa24e159ca28925d1
  hostname: MacBook-Pro-3
  iterations_since_restore: 98
  node_ip: 192.168.0.4
  pid: 36342
  time_since_restore: 117.63234782218933
  time_this_iter_s: 0.9451630115509033
  time_total_s: 117.63234782218933
  timestamp: 1616081617
  timesteps_since_restore: 0
  training_iteration: 98
  trial_id: 039ee_00002
  
Result for train_breast_cancer_039ee_00002:
  binary_error: 0.783220651505839
  date: 2021-03-18_15-33-42
  done: true
  experiment_id: bbd963254b9740aaa24e159ca28925d1
  hostname: MacBook-Pro-3
  iterations_since_restore: 101
  node_ip: 192.168.0.4
  pid: 36342
  time_since_restore: 122.57582998275757
  time_this_iter_s: 3.038051128387451
  time_total_s: 122.57582998275757
  timestamp: 1616081622
  timesteps_since_restore: 0
  training_iteration: 101
  trial_id: 039ee_00002
  


Trial name,status,loc,boosting_type,learning_rate,num_leaves,iter,total time (s),binary_error
train_breast_cancer_039ee_00000,TERMINATED,,gbdt,0.00018766,484,101,114.072,0.781592
train_breast_cancer_039ee_00001,TERMINATED,,dart,0.000536684,395,101,96.5724,0.785771
train_breast_cancer_039ee_00002,TERMINATED,,gbdt,4.10828e-07,934,101,122.576,0.783221
train_breast_cancer_039ee_00003,TERMINATED,,dart,0.0449364,558,101,113.198,0.831254


Trial name,status,loc,boosting_type,learning_rate,num_leaves,iter,total time (s),binary_error
train_breast_cancer_039ee_00000,TERMINATED,,gbdt,0.00018766,484,101,114.072,0.781592
train_breast_cancer_039ee_00001,TERMINATED,,dart,0.000536684,395,101,96.5724,0.785771
train_breast_cancer_039ee_00002,TERMINATED,,gbdt,4.10828e-07,934,101,122.576,0.783221
train_breast_cancer_039ee_00003,TERMINATED,,dart,0.0449364,558,101,113.198,0.831254


2021-03-18 15:33:42,303	INFO tune.py:450 -- Total run time: 131.87 seconds (131.08 seconds for the tuning loop).
Best hyperparameters found were:  {'objective': 'binary', 'metric': 'binary_error', 'verbose': -1, 'boosting_type': 'gbdt', 'num_leaves': 484, 'learning_rate': 0.0001876601217135646}
